In [39]:
from pytezos import pytezos
import requests
import json
from pandas import json_normalize
import time
from random import choice
from pprint import pprint
from random import randint

address = 'KT1CepDBrMg73d7LHm773KAsunjAjgLYituP'
# pytezos = pytezos.using(shell='https://florencenet-tezos.giganode.io')
pytezos = pytezos.using(shell='https://api.tez.ie/rpc/florencenet')

In [40]:
keys = dict(
    a = pytezos.using(key='test-keys/tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ.json').contract(address),
    b = pytezos.using(key='test-keys/tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos.json').contract(address),
    c = pytezos.using(key='test-keys/tz1ZAzDvkZCT2LAyPN8Kdxw3kes7xfWerZhZ.json').contract(address),
    d = pytezos.using(key='test-keys/tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE.json').contract(address),
    e = pytezos.using(key='test-keys/tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM.json').contract(address)
)

In [41]:
last_used = {}

def pick_random_free_key(keys, last_used, time_to_use_again=60):
    while True:
        random_key = choice(list(keys.keys()))
        current_time = time.time()
        elapsed_time = current_time - last_used.get(random_key, 0)
        if elapsed_time > time_to_use_again:
            last_used[random_key] = current_time
            return random_key
        else:
            time.sleep(1)

def wait_while_key_can_be_used(key, last_used):
    the_only_key = {key: keys[key]}
    return pick_random_free_key(the_only_key, last_used)

In [54]:
CURRENCY_PAIRS = [
    'BAT-USDC', 'BTC-USD', 'COMP-USD', 'DAI-USDC', 'ETH-USD',
    'KNC-USD', 'LINK-USD', 'REP-USD', 'ZRX-USD',
    # 'XTZ-USD', 
]

# CURRENCY_PAIRS = ['XTZ-USD']

TARGET_DYNAMICS = [
    1_000_000, 1_000_000, 1_000_000, 1_000_000, 1_000_000,
    950_000, 950_000, 950_000,
    1_050_000, 1_050_000, 1_050_000,
    900_000,
    1_000_000
]

RUN_TIME = int(time.time())

# picking nearest available hour:
RUN_TIME = RUN_TIME - RUN_TIME % 300
# BETS_CLOSE_TIME = [RUN_TIME + 1200]
BETS_CLOSE_TIME = [RUN_TIME + 604800*2]  # two weeks to collect bets
MEASURE_PERIOD = [300, 600, 1200]
LIQUIDITY_PERCENT = [10_000, 25_000, 50_000, 100_000]
MEASURE_START_FEE = [100_000, 200_000, 500_000]
EXPIRATION_FEE = [100_000, 200_000, 500_000]

LIQUIDITY_VALUE = [1_000_000, 2_000_000, 5_000_000, 10_000_000]
BET_VALUE = [100_000, 250_000, 500_000, 1_000_000]

EVENTS_COUNT = 5

In [55]:
def create_random_event():
    event_params = {
        'currencyPair': choice(CURRENCY_PAIRS),
        'targetDynamics': choice(TARGET_DYNAMICS),
        'betsCloseTime': choice(BETS_CLOSE_TIME),
        'measurePeriod': choice(MEASURE_PERIOD),
        'liquidityPercent': choice(LIQUIDITY_PERCENT),
    }

    key = pick_random_free_key(keys, last_used)
    print(f'creating event with {key}:')
    pprint(event_params)

    fees = (
        keys[key].storage()['config']['expirationFee']
        + keys[key].storage()['config']['measureStartFee'])

    keys[key].newEvent(event_params).with_amount(fees).as_transaction().autofill().sign().inject(_async=False)
    return event_params

def provide_liquidity(event_id):
    liquidity_value = choice(LIQUIDITY_VALUE)
    pl_params = {
        'eventId': event_id,
        'expectedRatioAboveEq': choice([1, 2, 3, 4]),
        'expectedRatioBelow': choice([1, 2, 3, 4]),
        'maxSlippage': 10_000_000_000,
    }

    key = pick_random_free_key(keys, last_used)
    print(f'providing liquidity for {event_id} with {key}:')

    transaction = keys[key].provideLiquidity(pl_params).with_amount(liquidity_value).as_transaction()
    return transaction.autofill().sign().inject(_async=False)


def bet(event_id):
    bet_value = choice(BET_VALUE)
    bet_choice = choice(['aboveEq', 'below'])
    bet_params = {
        'bet': bet_choice,
        'eventId': event_id,
        'minimalWinAmount': bet_value
    }

    key = pick_random_free_key(keys, last_used)
    print(f'bet "{bet_choice}" for {event_id} with {key}:')

    transaction = keys[key].bet(bet_params).with_amount(bet_value).as_transaction()
    return transaction.autofill().sign().inject(_async=False)

def start_measurement(event_id):
    key = pick_random_free_key(keys, last_used)
    print(f'starting measurement for {event_id} with {key}:')

    transaction = keys[key].startMeasurement(event_id).as_transaction()
    return transaction.autofill().sign().inject(_async=False)

def close(event_id):
    key = pick_random_free_key(keys, last_used)
    print(f'closing for {event_id} with {key}:')

    transaction = keys[key].close(event_id).as_transaction()
    return transaction.autofill().sign().inject(_async=False)

def withdraw(event_id, key):
    print(f'withdrawing for {event_id} with {key}:')

    params = {'eventId': event_id, 'participantAddress': keys[key].key.public_key_hash()}
    transaction = keys[key].withdraw(params).as_transaction()
    return transaction.autofill().sign().inject(_async=False)

def repeat_to_succeed_wrapper(func):
    def wrapped(*args, **kwargs):
        while True:
            try:
                return f(*args, **kwargs)
            except MichelsonError as e:
                print(f'catched MichelsonError {e}')
                time.sleep(1)
    return wrapped

def trigger_force_majeure(event_id):
    key = pick_random_free_key(keys, last_used)
    transaction = keys[key].triggerForceMajeure(event_id).as_transaction()
    transaction.autofill().sign().inject(_async=False)
    print(f'event: {event_id} triggered by key: {key}')

'''
def wait_until(time_until_wait):
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'
    return 'ready'
'''

"\ndef wait_until(time_until_wait):\n    while time.time() < time_until_wait:\n        time.sleep(1)\n        yield f'waiting until {time_until_wait}'\n    return 'ready'\n"

In [56]:
def repeat_to_succeed_wrapper(func):
    def wrapped(*args, **kwargs):
        while True:
            try:
                return f(*args, **kwargs)
            except MichelsonError as e:
                print(f'catched MichelsonError {e}')
                time.sleep(1)
    return wrapped

In [57]:
# event_params = keys['a'].storage['events'][6]()

In [73]:
ORACLE_DELAY = 10*60  # starting calling oracle after this time

# TODO: split event to actions, yield after each action, if action is not succeed - yield and then repeat action
def event_pipeline(event_id):
    '''
    event_params = create_random_event()
    yield event_params

    # waiting till event is created:
    # TODO: is it possible to move this kind of waiting to async func?
    # TODO: maybe I should learn how to use async instead?
    time_until_wait = time.time() + 90
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'


    # providing liquidity:
    yield provide_liquidity(event_id)

    time_until_wait = time.time() + 90
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'
    '''

    event_params = keys['a'].storage['events'][event_id]()
    actions = randint(10, 100)
    for action in range(actions):
        next_action = choice([provide_liquidity, bet, bet, bet])
        if time.time() + 90 > event_params['betsCloseTime']:
            break

        yield next_action(event_id)

    # waiting for bets close time:
    time_until_wait = event_params['betsCloseTime'] + ORACLE_DELAY
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'
    yield start_measurement(event_id)

    # waiting for close time:
    time_until_wait = event_params['betsCloseTime'] + ORACLE_DELAY*2 + event_params['measurePeriod']
    while time.time() < time_until_wait:
        time.sleep(1)
        yield f'waiting until {time_until_wait}'
    yield close(event_id)

    for key in keys:
        # blocking waiting (anyway no one can use keys if they in transactions):
        try:
            wait_while_key_can_be_used(key, last_used)
            yield withdraw(event_id, key)
        except Exception as e:
            print('error', e)

    yield 'finished'

In [74]:
next_event_id = keys['a'].storage['nextEventId']()
# next_event_id = 22
next_event_id

28

In [75]:
pipelines = [event_pipeline(k) for k in [25, 26, 27]]

In [ ]:
while pipelines:
    for n in range(len(pipelines)):
        try:
            next(pipelines[n])
        except StopIteration:
            # should work, but will skips one pipeline when popping:
            pipelines.pop(n)
        except IndexError:
            continue

In [293]:
for event_id in [6, 7, 8]:
    for key in keys:
        # blocking waiting (anyway no one can use keys if they in transactions):
        try:
            wait_while_key_can_be_used(key, last_used)
            withdraw(event_id, key)
        except Exception as e:
            print('error', e)

withdrawing for 6 with a:
withdrawing for 6 with b:
withdrawing for 6 with c:
withdrawing for 6 with d:
withdrawing for 6 with e:
withdrawing for 7 with a:
withdrawing for 7 with b:
withdrawing for 7 with c:
withdrawing for 7 with d:
withdrawing for 7 with e:
withdrawing for 8 with a:
withdrawing for 8 with b:
withdrawing for 8 with c:
withdrawing for 8 with d:
withdrawing for 8 with e:


In [28]:
response = requests.get(f'https://api.edo2net.tzkt.io/v1/contracts/{address}/storage/history?limit=100')
data = json.loads(response.text)
df = json_normalize(data)

### removeme:

In [1]:
import pandas as pd
import requests
import json

response = requests.get('https://api.better-call.dev/v1/contract/mainnet/KT1LRboPna9yQY9BrjtQYDS1DVxhKESK4VVd/tokens/holders?token_id=0')
data = json.loads(response.text)
df = pd.json_normalize(data).T

df = df.rename(columns={0: 'tokens'})
df.tokens = df.tokens.astype(int)
df['share'] = df.tokens / df.tokens.sum()
df = df.sort_values('share', ascending=False)
df['WRAP'] = df.tokens / 10**8

In [2]:
df.share.map(lambda x: f'{x*100:.3f}')[0:20]

tz1d4EXxyFQV23dGV8pQhan4PK3oLbjY5yoY    55.428
KT1FG63hhFtMEEEtmBSX2vuFmP87t9E7Ab4t    18.140
KT1J82G1XVAA4oqjC44qskihdWerrneNVHnn     3.381
KT1DQg57yTJ7QKxzk6AkPtDGvZYTM5pX4GYE     2.991
KT1APj29bNiMdrxQ2Nah49MH89TweW84nSXQ     2.826
tz1cNABC2qtbbHKDDKdvxRkcyopL1kEfbpgV     2.773
KT1UfeNuqT3F6ntYwucDGPHwHdYa1pQbWfST     2.205
KT1HVrGpv4GUoSR6qCmkxmHzAFtwfKxzfcop     2.193
KT1GCNfU4RQ85VgQF3fsj69Lgw8Ucz7RGoph     1.588
KT1KENVgwsrAnXZHkm5MptHWhMYiXeG5hwa3     1.443
tz1Mc61yajNASzh5KWZFtYs4i7EJbzAxQC4K     0.887
KT1UeEeeSfSd7uDWpbxBTBkBBpVeDrcqcUr7     0.745
KT1Stv6ATejBZ2uD9eMm5HyQ3nrNrC32zpRL     0.660
KT1UxnT4id9zKz1tgK1UxwpigFi5ny7vTCu9     0.614
KT1LXQT3b39wLSiFQ1rHatzALzeKsRGmd8gr     0.482
KT19ohHrCtSv5u4RzU3SySZJPKGhcovkf9sS     0.416
KT1NSKpB8ppqABLAEDozUmcLv3QUceN5uHQi     0.408
KT1PUFe6Gc2k8HKFXFEpVSoCVjM9uWUY54h3     0.250
KT1U9pATbqyDL3ZXzTQnPN2LnaGN78UnpzUw     0.238
KT1KY55T9jVZwFaCpRcj62LQNwoVYgtRHG6F     0.228
Name: share, dtype: object

In [3]:
total_supply = 10000000000000000
decimals = 10**8
df.tokens.sum() / total_supply

0.0677055358837802

In [4]:
current_supply = df.tokens.sum()
current_supply / decimals

6770553.58837802

In [5]:
22000*decimals / current_supply

0.0032493650205743965

In [6]:
token_price =  0.168

In [7]:
xtz_price = 2.44

In [8]:
total_supply / decimals * token_price / 10**6 * xtz_price

40.992

In [9]:
current_supply / decimals * token_price / 10**6 * xtz_price

2.775385326947918

In [10]:
year_supply = 44 * 10**6
year_supply * token_price / 10**6 * xtz_price

18.03648

### Quipuswap:

In [42]:
response = requests.get('https://api.better-call.dev/v1/contract/mainnet/KT1FG63hhFtMEEEtmBSX2vuFmP87t9E7Ab4t/tokens/holders?token_id=0')
data = json.loads(response.text)
df = pd.json_normalize(data).T

In [43]:
df = df.rename(columns={0: 'tokens'})
df.tokens = -df.tokens.astype(int)
df = df[ df.tokens > 0 ]
df['share'] = df.tokens / df.tokens.sum()
df = df.sort_values('share', ascending=False)
df['QUIPU'] = df.tokens / 10**6

In [45]:
df.iloc[:25]

,tokens,share,QUIPU
tz1cmanHzitf3UgWgECzLpGUKpzEfmGZaFsw,15108020407,0.091118,15108.020407
tz1L8m6MGdvK2WA65wQ7DeVwXVmCS72JKWeN,13862055056,0.083603,13862.055056
tz1gJpwrJy66fWH2niNwLzWv2YhbggKq8crT,11251481661,0.067859,11251.481661
tz1XYkmAHtkZFjZLdWNoL2BTqUws4oD4H1PR,10743667679,0.064796,10743.667679
tz1Yu1qh4mk9ejf2MLxRNnRwdHoidXqoSqqm,8832254547,0.053268,8832.254547
tz1W19DveaP3mW4WvFtdVdqGsWwBL5q1QDSK,7290406374,0.043969,7290.406374
tz1ZNxMa8XsMa8JTv776X9iCETcNoo4SY629,6702908997,0.040426,6702.908997
tz1PaJwmmL2nrRt5K6HvwFEc6fUfxNe6Dyp5,6100002365,0.036790,6100.002365
tz1bAUF7TenZVtYEysqdDcvP7XiA3n35jVEN,6025997065,0.036343,6025.997065
tz1QGZU9mpkSYkMVkoi2J22RtoqMY7RRzjjn,5536919617,0.033394,5536.919617
